In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
from collections import defaultdict
import numpy as np
import pandas as pd
import bitsandbytes
import accelerate
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.datageneration import Prompter
print(f"accelerate={accelerate.__version__}, bitsandbytes={bitsandbytes.__version__}")

accelerate=0.29.2, bitsandbytes=0.43.1


In [2]:
target_size = 30*10
model_dir = Path("/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1")
model_max_length = 4096
seed = 420

In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(seed)
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [4]:
with open("input/student_names.json") as f:
    student_names = set(json.load(f))
print(f"{len(student_names):,} student names")

104 student names


In [5]:
df = pd.read_csv("input/geocode-db.csv", low_memory=False, names=["street","lat","lng"])
street_addresses = [x.title() for x in df["street"]]
print(f"{len(street_addresses):,} street addresses={street_addresses[:10]}")
street_addresses = set(street_addresses)
df.info()

11,562 street addresses=['Adis Road (Liv On Sophia)', 'Adis Road (Parc Sophia)', 'Admiralty Drive (Block 353A)', 'Admiralty Drive (Block 353B)', 'Admiralty Drive (Block 353C)', 'Admiralty Drive (Block 354A)', 'Admiralty Drive (Block 354C)', 'Admiralty Drive (Block 354D)', 'Admiralty Drive (Block 356B)', 'Admiralty Drive (Block 356C)']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11562 entries, 0 to 11561
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   street  11562 non-null  object 
 1   lat     11562 non-null  float64
 2   lng     11562 non-null  float64
dtypes: float64(2), object(1)
memory usage: 271.1+ KB


In [6]:
tokenizer = AutoTokenizer.from_pretrained(str(model_dir), model_max_length=model_max_length)
tokenizer.pad_token = tokenizer.eos_token
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")
print(f"pad_token_id={tokenizer.pad_token_id}")

LlamaTokenizerFast(name_or_path='/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'attention_mask']
pad_token_id=2


In [7]:
%%time
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    str(model_dir),
    device_map = "auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

CPU times: user 4min 4s, sys: 3min 29s, total: 7min 34s
Wall time: 14min 3s


In [8]:
print(model)

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (w3): Linear4bit(in_features=4096, out_

In [9]:
prompter = Prompter(target_size=target_size, student_names=student_names, street_addresses=street_addresses)
res = prompter.prompt()
print(res.prompt)

[INST] Write a short essay on the following topic:
        In Education Policy and Reform, how can the integration of a learning launch tool within the design thinking process enhance the development and testing of ideas, prototypes, and products?
        Intersperse the following information in different parts of the essay:
        1. Give an example that occurred near Bukit Panjang Ring Road (Block 235), at a school frequented by Shawn Duncan
        2. Give an example citing someone named ShawnDuncan
        3. Give an example citing: periscope.tv/officialshawniamduncan [/INST]
        


In [10]:
%%time
rows = []
for _ in tqdm(range(target_size)):
    res = prompter.prompt()
    inputs = tokenizer(res.prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
        top_k=100,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    generated_text = generated_text.split("[/INST]")[1]
    generated_text = generated_text.strip()
    rows.append({
        "text": generated_text,
        "prompt": res.prompt,
        "street_address": res.street_address,
        "student_name": res.student_name,
        "username": res.username,
        "personal_url": res.personal_url,
        "model": model_dir.name,
    })

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [8:57:32<00:00, 107.51s/it]

CPU times: user 8h 26min 28s, sys: 31min 16s, total: 8h 57min 45s
Wall time: 8h 57min 32s


In [11]:
df = pd.DataFrame.from_records(rows)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            300 non-null    object
 1   prompt          300 non-null    object
 2   street_address  300 non-null    object
 3   student_name    300 non-null    object
 4   username        300 non-null    object
 5   personal_url    300 non-null    object
 6   model           300 non-null    object
dtypes: object(7)
memory usage: 16.5+ KB


In [12]:
df.head(10)

,text,prompt,street_address,student_name,username,personal_url,model
0,"In the field of architecture and urban design, visualization tools have become increasingly important for fostering innovation and collaboration in a design thinking approach. These tools allow architects and designers to create detailed representations of their ideas, enabling them to better understand and refine their designs before they are built. Moreover, these tools also make it easier for stakeholders to understand and provide feedback on proposed designs, which can help improve the final product.\n\nOne example of how visualization tools have been used effectively in this way can be seen near Hougang Central (Block 837) in a nature park frequently visited by Meinrad Foster. Here, urban planners used virtual reality technology to create immersive simulations of various design options for the park. This allowed Meinrad and other community members to explore and evaluate the proposed changes in a more intuitive and engaging way than traditional 2D drawings or models would have provided. By gathering input from the community through this interactive process, the designers were able to incorporate local perspectives into the project, resulting in a park that better met the needs and desires of the people who use it every day.\n\nAnother example of the power of visualization tools in architecture and urban design comes from the work of designer known as ""_4807.meinrad-iam--foster"". Using advanced software, Meinrad created detailed renderings of a proposed residential complex that highlighted its innovative features, such as green roofs and communal spaces. By sharing these images with colleagues, clients, and investors, Meinrad was able to generate excitement around the project and build support for it among key decision makers. This collaborative approach ultimately led to the successful construction of a building that not only looks stunning but also addresses many of the challenges posed by contemporary urban living.\n\nFinally, there is the case of Meinrad Foster's live streaming channel on Periscope (periscope.tv/meinradfosterxxx). Through his live streams, Meinrad gives viewers real-time updates on ongoing projects, sharing sketches, models, and other visual materials that showcase his ideas and design processes. By using this platform to engage with a wider audience, Meinrad has not only gained valuable feedback and insights from fans and followers, but he has also inspired others to think creatively about the potential of architecture and urban design to enhance our daily lives.\n\nIn conclusion, visualization tools are essential for architects and urban designers seeking","[INST] Write a short essay on the following topic:\n In Architecture and Urban Design, how can visualization tools foster innovation and collaboration in a design thinking approach?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Hougang Central (Block 837), at a nature park frequented by Meinrad Foster\n 2. Give an example citing someone named _4807.meinrad-iam--foster\n 3. Give an example citing: periscope.tv/meinradfosterxxx [/INST]\n",Hougang Central (Block 837),Meinrad Foster,_4807.meinrad-iam--foster,periscope.tv/meinradfosterxxx,Mixtral-8x7B-Instruct-v0.1
1,"In sustainable urban development, the design thinking process plays a crucial role in fostering innovation and collaboration. This process involves empathizing with users, defining their needs and problems, ideating potential solutions, prototyping those solutions, and testing them in real-world scenarios. One powerful tool that can be used within this process is storytelling. By using stories to illustrate and communicate user needs and perspectives, designers can better understand these individuals and create more effective, sustainable urban solutions.\n\nA great example of storytelling being used in sustainable urban development can be seen near 

In [13]:
df.to_parquet("output/part.parquet")
assert df.notna().all(axis=None)

In [14]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 9:11:37.162350
